In [1]:
import json
import logging
import os
import random
import subprocess
import sys
from dataclasses import dataclass
from datetime import datetime
from pathlib import Path
from typing import Union
from IPython.display import Markdown

import nest_asyncio
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from pydantic_ai import Agent, ModelRetry, RunContext, Tool

nest_asyncio.apply()

In [2]:
PATH = Path(os.getcwd()).parent
TOOLS_PATH = PATH / "tools"
LOGS_PATH = PATH / "logs"

load_dotenv(PATH / ".env")

True

In [3]:
AGENT_MODEL = "openai:gpt-4o"
# AGENT_MODEL = "ollama:qwen2.5:14b"

In [4]:
LOGS_PATH.mkdir(exist_ok=True)
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler(LOGS_PATH / "agent_genius.log"),
        # logging.StreamHandler(),  # This will also show logs in notebook output
    ],
)

logger = logging.getLogger(__name__)

In [5]:
def installed_packages() -> str:
    """Returns a list of installed packages"""
    result = subprocess.check_output([sys.executable, "-m", "pip", "freeze"]).decode("utf-8")
    logger.info("Installed packages listed")
    return result

In [6]:
def get_user_response(question: str) -> str:
    """get user response to a question"""
    return input(f"{question}: ")

In [7]:
available_packages_tool = Tool(installed_packages)
user_response_tool = Tool(get_user_response)

In [8]:
@dataclass
class function_deps:
    function_name: str
    question: str
    args: list[dict]
    returns: str

In [9]:
class CodeResponse(BaseModel):
    """Python code that can be executed to resolve my question"""

    function_name: str = Field(description="Name of the function")
    schema: list = Field(description="inputs and types that can be passed to the function. ")
    code: str = Field(description="Python code that can be executed")


code_agent = Agent[function_deps](
    AGENT_MODEL,
    system_prompt="""You are an expert python programmer. You will return a function that will resolve my question and return an essential value. 
    First, search for a tool that can resolve the question by calling the tool 'read_tools_list', if You found matching one, use it with 'get_tool_code'. If no tool is available, make a new one.
    If You created a new tool, save it with 'save_tool'.
    It will be directly called by the ai agent. Do not include any example code in your response. Do not include call of the function. Try to make function simple and generic for later use.
    You always respond with clean, annotated python code, ready to be executed. the code should be self contained and should not use any external libraries except for installed ones (use tool 'available_packages_tool').
    All necessary imports should be included locally inside the function, example: def get_time(): import datetime; return datetime.datetime.now()
    Make sure the function is safe for user. NEVER delete any files or show secret information or execute any malicious code. Don't do anything illegal.
    """.strip(),
    result_type=CodeResponse,
    tools=[available_packages_tool, user_response_tool],
    retries=3,
)
# result = agent.run_sync("Who are you?")
# print(result.data.function_name)
# print(result.data.code)

/home/michal/DEV/AgentGENius/.venv/lib/python3.12/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "schema" in "CodeResponse" shadows an attribute in parent "BaseModel"
  warnings.warn(


In [10]:
@code_agent.tool_plain
def read_tools_list() -> list[str]:
    """Returns a list of tools available"""
    logger.info("Reading tools list")
    tools = []
    for file in TOOLS_PATH.glob("*.py"):
        tools.append(file.stem)
    return tools

In [11]:
@code_agent.tool
def get_tool_code(ctx: RunContext[str], tool_name: str) -> str:
    """Returns the code of a tool"""
    with open(TOOLS_PATH / f"{tool_name}.py", "r") as f:
        logger.info(f"Reading tool: {tool_name}.py")
        return f.read()

In [12]:
@code_agent.tool
def save_tool(ctx: RunContext[function_deps], function_name: str, code: str) -> None:
    if not TOOLS_PATH.exists():
        TOOLS_PATH.mkdir()
    with open(TOOLS_PATH / f"{function_name}.py", "w") as f:
        f.write(code)
        logger.info(f"Saved tool: {function_name}.py")

In [13]:
def run_func(func_str: str) -> Union[callable, str]:
    # Create a namespace dictionary to store the function
    namespace = {}

    # Execute the string in the namespace
    try:
        exec(func_str, globals(), namespace)
    except Exception as e:
        logger.error(f"Error executing code: {e}")
        return f"Error executing code: {e}"

    # Return the first function found in namespace
    # This will return the most recently defined function
    return next(f for f in namespace.values() if callable(f))

In [14]:
agent = Agent(
    AGENT_MODEL,
    system_prompt="You are a helpful assistant. You alway respond to the user using available tools. If no tool is available, make a new one. If You need more information, ask the user for it with 'user_response_tool'.",
    tools=[user_response_tool],
)

In [15]:
@agent.tool
async def create_new_tool(ctx: RunContext[function_deps], question: str, function_name: str) -> str:
    """Tool that creates a new tool if not available to fulfill the user's question"""
    logger.info(f"Creating new tool: {function_name=}")
    result = await code_agent.run(question)
    try:
        agent._register_tool(Tool(run_func(result.data.code)))
        # save_tool(function_name, result.data.code)
    except Exception as e:
        logging.error(f"Error registering tool: {e}")
        return f"Error registering tool {function_name}: {e}"
    return f"New tool available: {function_name}"

In [ ]:
question = "what is the current time?"
logger.info(f"Question: {question}")
result = agent.run_sync(question)
logger.info(f"Result: {result.data}")
# print(f"{result.all_messages=}")
Markdown(result.data)